# Project Review Grading

### Version: 0.0.2
#### Update: 11.15.2023 Changed method to extract group numbers using regular expression.

Author: Jason Chen

## How to use this script:

1. Go to Canvas, and generate the personal token, paste in the cell below.
    - To do so, go to canvas.ucsd.edu, Account -> Settings -> Approved Integrations -> + New Access Token. Generate the access token, set the purpose and the expiration date, copy it.
2. Get the course id and assignment id, paste in the cell below.
3. Download the responses for Project Review as a csv file, upload it under the same directory as this script.
4. Paste the file path and the column name containing the group numbers in the cell below.
5. Run all the cells to publish score.

## Note:
- When setting up the assignment on Canvas, please select "This is a Group Assignment" and also select "Assign Grades to Each Student Individually".

In [20]:
# Import packages needed:

import pandas as pd
import re
import requests

In [21]:
# Initialize the variables:

# The URL of Canvas api, no need to change.
API_URL = "https://canvas.ucsd.edu"

# The access token for Canvas, need to generate on Canvas website and paste here.
API_KEY = "13171~m5qrWdND5GevNLg9WfeHzoY8pghyfZQWUT7mDo5G4npFEXvLucRuNChhzZ9HKxIG"

# The course id, can be found on Canvas.
COURSE_ID = "48733"

# The assignment id, can be found on Canvas.
ASSIGNMENT_ID = "693725"

# The score for the assignmnet Project Review, can be changed accordingly.
SCORE = 5

# The path of csv file contains responses for Project Review.
file_path = "COGS 108 Previous Project Review (Fa23) (Responses) - Form Responses 1.csv"

# The column name that contains the group numbers.
column_name = "What is your group name? Make sure you enter it correctly or you may not get credit, e.g. Group007_Fa23"

In [22]:
# Extract group numbers that submitted the project review.

def extract_group_numbers_case_insensitive_from_csv(file_path, column_name):
    df = pd.read_csv(file_path)
    group_numbers = set()
    
    # Find patterns that contains the group number.
    # Example patterns: "group106_Fa23", "group_106_Fa23", "Group 106_Fa23", or "Group 106"
    pattern = re.compile(r'group_?\s?(\d+)_?', re.IGNORECASE)

    for item in df[column_name].dropna():
        match = pattern.search(item)
        if match:
            group_number = match.group(1)
            group_numbers.add(group_number)

    return group_numbers

# Usage

unique_group_numbers = extract_group_numbers_case_insensitive_from_csv(file_path, column_name)
print(unique_group_numbers)

{'097', '002', '085', '088', '121', '030', '042', '036', '032', '003', '062', '084', '047', '081', '107', '021', '028', '143', '065', '052', '095', '123', '117', '141', '099', '142', '025', '073', '013', '031', '130', '129', '044', '033', '055', '087', '019', '071', '074', '111', '005', '039', '133', '114', '057', '137', '091', '131', '010', '122', '016', '110', '022', '067', '040', '086', '146', '068', '109', '061', '024', '077', '096', '127', '007', '048', '132', '070', '053', '064', '102', '046', '060', '112', '082', '104', '128', '027', '083', '072', '092', '093', '066', '020', '050', '018', '026', '051', '017', '090', '063', '098', '103', '101', '135', '113', '008', '041', '009', '138', '148', '100', '043', '058', '045', '079', '015', '139', '035', '106', '125', '120', '147', '037', '105', '118', '115', '080', '059', '006', '140', '069', '076', '011', '054', '124', '134', '023', '094', '075', '136', '034', '056', '014', '078', '144', '108', '126', '029', '001', '116', '038', '012'

In [23]:
len(unique_group_numbers)

147

In [26]:
sorted(unique_group_numbers)

['001',
 '002',
 '003',
 '004',
 '005',
 '006',
 '007',
 '008',
 '009',
 '010',
 '011',
 '012',
 '013',
 '014',
 '015',
 '016',
 '017',
 '018',
 '019',
 '020',
 '021',
 '022',
 '023',
 '024',
 '025',
 '026',
 '027',
 '028',
 '029',
 '030',
 '031',
 '032',
 '033',
 '034',
 '035',
 '036',
 '037',
 '038',
 '039',
 '040',
 '041',
 '042',
 '043',
 '044',
 '045',
 '046',
 '047',
 '048',
 '049',
 '050',
 '051',
 '052',
 '053',
 '054',
 '055',
 '056',
 '057',
 '058',
 '059',
 '060',
 '061',
 '062',
 '063',
 '064',
 '065',
 '066',
 '067',
 '068',
 '069',
 '070',
 '071',
 '072',
 '073',
 '074',
 '075',
 '076',
 '077',
 '078',
 '079',
 '080',
 '081',
 '082',
 '083',
 '084',
 '085',
 '086',
 '087',
 '088',
 '090',
 '091',
 '092',
 '093',
 '094',
 '095',
 '096',
 '097',
 '098',
 '099',
 '100',
 '101',
 '102',
 '103',
 '104',
 '105',
 '106',
 '107',
 '108',
 '109',
 '110',
 '111',
 '112',
 '113',
 '114',
 '115',
 '116',
 '117',
 '118',
 '119',
 '120',
 '121',
 '122',
 '123',
 '124',
 '125',
 '126',


In [24]:
# Get group ids for each group:

def get_group_ids(course_id, api_url, api_key):
    endpoint = f"{api_url}/api/v1/courses/{course_id}/groups"
    headers = {"Authorization": f"Bearer {api_key}"}
    group_ids = {}

    while endpoint:
        response = requests.get(endpoint, headers=headers)
        if response.status_code == 200:
            groups = response.json()
            for group in groups:
                group_number = group['name'].lower().split('_')[0][5:]
                group_ids[group_number] = group['id']

            links = response.links
            endpoint = links['next']['url'] if 'next' in links else None
        else:
            print("Failed to retrieve groups: ", response.text)
            break

    return group_ids

# Usage

group_ids = get_group_ids(COURSE_ID, API_URL, API_KEY)

In [25]:
group_ids

{'001': 133709,
 '002': 133710,
 '003': 133711,
 '004': 133712,
 '005': 133713,
 '006': 133714,
 '007': 133715,
 '008': 133716,
 '009': 133717,
 '010': 133718,
 '011': 133719,
 '012': 133720,
 '013': 133721,
 '014': 133722,
 '015': 133723,
 '016': 133724,
 '017': 133725,
 '018': 133726,
 '019': 133727,
 '020': 133728,
 '021': 133729,
 '022': 133730,
 '023': 133731,
 '024': 133732,
 '025': 133733,
 '026': 133734,
 '027': 133735,
 '028': 133736,
 '029': 133737,
 '030': 133738,
 '031': 133739,
 '032': 133740,
 '033': 133741,
 '034': 133742,
 '035': 133743,
 '036': 133744,
 '037': 133745,
 '038': 133746,
 '039': 133747,
 '040': 133748,
 '041': 133749,
 '042': 133750,
 '043': 133751,
 '044': 133752,
 '045': 133753,
 '046': 133754,
 '047': 133755,
 '048': 133756,
 '049': 133757,
 '050': 133758,
 '051': 133759,
 '052': 133760,
 '053': 133761,
 '054': 133762,
 '055': 133763,
 '056': 133764,
 '057': 133765,
 '058': 133766,
 '059': 133767,
 '060': 133768,
 '061': 133769,
 '062': 133770,
 '063': 

In [6]:
# Get student ids in each group for grading:

def get_student_ids_for_groups(course_id, group_ids, api_url, api_key):
    headers = {"Authorization": f"Bearer {api_key}"}
    student_ids = {}
    for group_number, group_id in group_ids.items():
        endpoint = f"{api_url}/api/v1/groups/{group_id}/users"
        response = requests.get(endpoint, headers=headers)
        if response.status_code == 200:
            students = response.json()
            student_ids[group_number] = [student['id'] for student in students]
        else:
            print(f"Failed to retrieve students for group ID {group_id}: {response.text}")
    return student_ids

In [7]:
student_ids = get_student_ids_for_groups(COURSE_ID, group_ids, API_URL, API_KEY)

In [8]:
student_ids

{'001': [108648, 96131, 137403, 26607, 130720],
 '002': [99002, 24678, 166914, 136784, 108148],
 '003': [171654, 129223, 107429, 158104, 96199],
 '004': [130947, 99725, 15279, 130922, 128524],
 '005': [170014, 128109, 99596, 133356, 133913],
 '006': [170628, 100403, 124614, 97278, 134241],
 '007': [133353, 136327, 129735, 98083, 136884],
 '008': [104952, 101920, 133124, 95337, 165074],
 '009': [169672, 170314, 131330, 129817, 129236],
 '010': [132711, 107541, 96896, 100730, 132071],
 '011': [95700, 170488, 96754, 136616, 129582],
 '012': [100638, 100805, 45393, 97451, 99571],
 '013': [97951, 168241, 100011, 95013, 166846],
 '014': [170159, 135159, 169677, 169754, 166937],
 '015': [136823, 135203, 105081, 97105, 135976],
 '016': [100907, 126389, 99861, 135820, 106490],
 '017': [131086, 84206, 168256, 134401, 106672],
 '018': [104930, 107444, 132190, 136685, 134875],
 '019': [108019, 97103, 167191, 127412, 97414],
 '020': [66006, 100900, 132220, 107645, 107315],
 '021': [167122, 100048, 

In [9]:
# Publish scores for each student:

def publish_scores(course_id, assignment_id, student_ids, unique_group_numbers, api_url, api_key, score):
    headers = {"Authorization": f"Bearer {api_key}"}

    for group_number in sorted(unique_group_numbers):
        ids = student_ids.get(group_number)
        if ids:
            score = score
            for student_id in ids:
                endpoint = f"{api_url}/api/v1/courses/{course_id}/assignments/{assignment_id}/submissions/{student_id}"
                payload = {"submission": {"posted_grade": score}}
                response = requests.put(endpoint, headers=headers, json=payload)

                if response.status_code == 200:
                    print(f"Score {score} published for student ID {student_id} in group {group_number}.")
                else:
                    print(f"Failed to publish score for student ID {student_id}: {response.text}")

In [11]:
publish_scores(COURSE_ID, ASSIGNMENT_ID, student_ids, unique_group_numbers, API_URL, API_KEY, SCORE)

Score 5 published for student ID 108648 in group 001
Score 5 published for student ID 96131 in group 001
Score 5 published for student ID 137403 in group 001
Score 5 published for student ID 26607 in group 001
Score 5 published for student ID 130720 in group 001
Score 5 published for student ID 99002 in group 002
Score 5 published for student ID 24678 in group 002
Score 5 published for student ID 166914 in group 002
Score 5 published for student ID 136784 in group 002
Score 5 published for student ID 108148 in group 002
Score 5 published for student ID 171654 in group 003
Score 5 published for student ID 129223 in group 003
Score 5 published for student ID 107429 in group 003
Score 5 published for student ID 158104 in group 003
Score 5 published for student ID 96199 in group 003
Score 5 published for student ID 130947 in group 004
Score 5 published for student ID 99725 in group 004
Score 5 published for student ID 15279 in group 004
Score 5 published for student ID 130922 in group 004


Score 5 published for student ID 92025 in group 032
Score 5 published for student ID 96286 in group 032
Score 5 published for student ID 130313 in group 033
Score 5 published for student ID 163522 in group 033
Score 5 published for student ID 105241 in group 033
Score 5 published for student ID 99424 in group 033
Score 5 published for student ID 128257 in group 033
Score 5 published for student ID 134050 in group 034
Score 5 published for student ID 105419 in group 034
Score 5 published for student ID 124815 in group 034
Score 5 published for student ID 93612 in group 034
Score 5 published for student ID 95712 in group 034
Score 5 published for student ID 100344 in group 035
Score 5 published for student ID 102965 in group 035
Score 5 published for student ID 96744 in group 035
Score 5 published for student ID 133359 in group 035
Score 5 published for student ID 103047 in group 035
Score 5 published for student ID 129102 in group 036
Score 5 published for student ID 99005 in group 036


Score 5 published for student ID 127796 in group 066
Score 5 published for student ID 130681 in group 066
Score 5 published for student ID 108717 in group 067
Score 5 published for student ID 107686 in group 067
Score 5 published for student ID 103513 in group 067
Score 5 published for student ID 95289 in group 067
Score 5 published for student ID 140172 in group 067
Score 5 published for student ID 135973 in group 068
Score 5 published for student ID 170938 in group 068
Score 5 published for student ID 104765 in group 068
Score 5 published for student ID 30341 in group 068
Score 5 published for student ID 131877 in group 068
Score 5 published for student ID 134098 in group 069
Score 5 published for student ID 126417 in group 069
Score 5 published for student ID 102630 in group 069
Score 5 published for student ID 129639 in group 069
Score 5 published for student ID 171822 in group 069
Score 5 published for student ID 134113 in group 070
Score 5 published for student ID 129728 in group

Score 5 published for student ID 170551 in group 099
Score 5 published for student ID 130744 in group 100
Score 5 published for student ID 129309 in group 100
Score 5 published for student ID 132336 in group 100
Score 5 published for student ID 99841 in group 100
Score 5 published for student ID 170865 in group 100
Score 5 published for student ID 125371 in group 101
Score 5 published for student ID 128375 in group 101
Score 5 published for student ID 134012 in group 101
Score 5 published for student ID 107802 in group 101
Score 5 published for student ID 108445 in group 101
Score 5 published for student ID 134440 in group 102
Score 5 published for student ID 98313 in group 102
Score 5 published for student ID 134529 in group 102
Score 5 published for student ID 104848 in group 102
Score 5 published for student ID 126395 in group 102
Score 5 published for student ID 100151 in group 103
Score 5 published for student ID 134467 in group 103
Score 5 published for student ID 133727 in group

Score 5 published for student ID 107634 in group 132
Score 5 published for student ID 139361 in group 132
Score 5 published for student ID 131066 in group 132
Score 5 published for student ID 107746 in group 132
Score 5 published for student ID 127733 in group 132
Score 5 published for student ID 130921 in group 133
Score 5 published for student ID 98862 in group 133
Score 5 published for student ID 127861 in group 133
Score 5 published for student ID 173393 in group 133
Score 5 published for student ID 135711 in group 133
Score 5 published for student ID 133401 in group 134
Score 5 published for student ID 101556 in group 134
Score 5 published for student ID 107369 in group 134
Score 5 published for student ID 135797 in group 134
Score 5 published for student ID 108552 in group 134
Score 5 published for student ID 98371 in group 135
Score 5 published for student ID 129664 in group 135
Score 5 published for student ID 101983 in group 135
Score 5 published for student ID 136454 in group